# HAR CNN training 

In [439]:
# Imports
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score
import matplotlib.pyplot as plt
%matplotlib inline

In [440]:
%run utils/utilities.py

In [441]:
class_label_vn = [u"Cổ tay",u"Cổ chân",u"Bả vai",u"Xoay người",u"Xoay đầu gối",u"Đi bộ",u"Chạy",u"Đạp xe",u"Đánh răng",u"Rửa tay",u"Thái",u"Trộn",u"Lên cầu thang",u"Xuống cầu thang"]

## Prepare data

In [442]:
rootDatasetDir = "./datasets/PTIT/normalized"
X_train, labels_train, list_ch_train = read_data(data_path="./datasets/PTIT/normalized", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path=rootDatasetDir, split="test") # test
assert list_ch_train == list_ch_test, "Mistmatch in channels!"

x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr
x_sensor_acc
x_watch_acc
x_watch_gyr
y_sensor_acc
y_watch_acc
y_watch_gyr
z_sensor_acc
z_watch_acc
z_watch_gyr


In [443]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [444]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [445]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [446]:
# Imports
import tensorflow as tf

### Hyperparameters

In [447]:
batch_size = 600       # Batch size
seq_len = WINDOWN_SIZE          # Number of steps or window size
learning_rate = 0.0001
epochs = 1000

n_classes = NUM_CLASS
n_channels = NUM_CHANNEL
print "n_channels %d" % n_channels

n_channels 9


### Construct the graph
Placeholders

In [448]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

In [449]:
with graph.as_default():
    # (batch, 128, 9) --> (batch, 64, 18)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=18, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 64, 18) --> (batch, 32, 36)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=36, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=72, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
    conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=144, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Now, flatten and pass to the classifier

In [450]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_4, (-1, 8*144))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [ ]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    print "num epoch %d" % epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

num epoch 1000
('Epoch: 0/1000', 'Iteration: 5', 'Train loss: 2.795125', 'Train acc: 0.068333')
('Epoch: 1/1000', 'Iteration: 10', 'Train loss: 2.758242', 'Train acc: 0.063333')
('Epoch: 1/1000', 'Iteration: 10', 'Validation loss: 2.728289', 'Validation acc: 0.068333')
('Epoch: 2/1000', 'Iteration: 15', 'Train loss: 2.714400', 'Train acc: 0.086667')
('Epoch: 3/1000', 'Iteration: 20', 'Train loss: 2.691210', 'Train acc: 0.103333')
('Epoch: 3/1000', 'Iteration: 20', 'Validation loss: 2.657622', 'Validation acc: 0.103333')
('Epoch: 4/1000', 'Iteration: 25', 'Train loss: 2.657260', 'Train acc: 0.093333')
('Epoch: 5/1000', 'Iteration: 30', 'Train loss: 2.634910', 'Train acc: 0.078333')
('Epoch: 5/1000', 'Iteration: 30', 'Validation loss: 2.604454', 'Validation acc: 0.105000')
('Epoch: 6/1000', 'Iteration: 35', 'Train loss: 2.613173', 'Train acc: 0.083333')
('Epoch: 7/1000', 'Iteration: 40', 'Train loss: 2.590060', 'Train acc: 0.096667')
('Epoch: 7/1000', 'Iteration: 40', 'Validation loss: 2

('Epoch: 64/1000', 'Iteration: 325', 'Train loss: 1.299572', 'Train acc: 0.548333')
('Epoch: 65/1000', 'Iteration: 330', 'Train loss: 1.302333', 'Train acc: 0.553333')
('Epoch: 65/1000', 'Iteration: 330', 'Validation loss: 1.258500', 'Validation acc: 0.603333')
('Epoch: 66/1000', 'Iteration: 335', 'Train loss: 1.274059', 'Train acc: 0.586667')
('Epoch: 67/1000', 'Iteration: 340', 'Train loss: 1.270601', 'Train acc: 0.606667')
('Epoch: 67/1000', 'Iteration: 340', 'Validation loss: 1.226141', 'Validation acc: 0.611667')
('Epoch: 68/1000', 'Iteration: 345', 'Train loss: 1.265838', 'Train acc: 0.573333')
('Epoch: 69/1000', 'Iteration: 350', 'Train loss: 1.231720', 'Train acc: 0.600000')
('Epoch: 69/1000', 'Iteration: 350', 'Validation loss: 1.198595', 'Validation acc: 0.615000')
('Epoch: 70/1000', 'Iteration: 355', 'Train loss: 1.245465', 'Train acc: 0.583333')
('Epoch: 71/1000', 'Iteration: 360', 'Train loss: 1.240767', 'Train acc: 0.590000')
('Epoch: 71/1000', 'Iteration: 360', 'Validati

('Epoch: 127/1000', 'Iteration: 640', 'Train loss: 0.747453', 'Train acc: 0.756667')
('Epoch: 127/1000', 'Iteration: 640', 'Validation loss: 0.747116', 'Validation acc: 0.766667')
('Epoch: 128/1000', 'Iteration: 645', 'Train loss: 0.730669', 'Train acc: 0.760000')
('Epoch: 129/1000', 'Iteration: 650', 'Train loss: 0.719910', 'Train acc: 0.738333')
('Epoch: 129/1000', 'Iteration: 650', 'Validation loss: 0.739059', 'Validation acc: 0.771667')
('Epoch: 130/1000', 'Iteration: 655', 'Train loss: 0.724846', 'Train acc: 0.758333')
('Epoch: 131/1000', 'Iteration: 660', 'Train loss: 0.718745', 'Train acc: 0.760000')
('Epoch: 131/1000', 'Iteration: 660', 'Validation loss: 0.731123', 'Validation acc: 0.775000')
('Epoch: 132/1000', 'Iteration: 665', 'Train loss: 0.745126', 'Train acc: 0.753333')
('Epoch: 133/1000', 'Iteration: 670', 'Train loss: 0.685782', 'Train acc: 0.798333')
('Epoch: 133/1000', 'Iteration: 670', 'Validation loss: 0.722695', 'Validation acc: 0.770000')
('Epoch: 134/1000', 'Iter

('Epoch: 189/1000', 'Iteration: 950', 'Train loss: 0.488438', 'Train acc: 0.853333')
('Epoch: 189/1000', 'Iteration: 950', 'Validation loss: 0.568084', 'Validation acc: 0.830000')
('Epoch: 190/1000', 'Iteration: 955', 'Train loss: 0.476345', 'Train acc: 0.858333')
('Epoch: 191/1000', 'Iteration: 960', 'Train loss: 0.503295', 'Train acc: 0.831667')
('Epoch: 191/1000', 'Iteration: 960', 'Validation loss: 0.564979', 'Validation acc: 0.826667')
('Epoch: 192/1000', 'Iteration: 965', 'Train loss: 0.514752', 'Train acc: 0.851667')
('Epoch: 193/1000', 'Iteration: 970', 'Train loss: 0.511522', 'Train acc: 0.858333')
('Epoch: 193/1000', 'Iteration: 970', 'Validation loss: 0.564553', 'Validation acc: 0.830000')
('Epoch: 194/1000', 'Iteration: 975', 'Train loss: 0.494587', 'Train acc: 0.840000')
('Epoch: 195/1000', 'Iteration: 980', 'Train loss: 0.478753', 'Train acc: 0.860000')
('Epoch: 195/1000', 'Iteration: 980', 'Validation loss: 0.559034', 'Validation acc: 0.823333')
('Epoch: 196/1000', 'Iter

('Epoch: 250/1000', 'Iteration: 1255', 'Train loss: 0.359462', 'Train acc: 0.885000')
('Epoch: 251/1000', 'Iteration: 1260', 'Train loss: 0.376361', 'Train acc: 0.895000')
('Epoch: 251/1000', 'Iteration: 1260', 'Validation loss: 0.478979', 'Validation acc: 0.858333')
('Epoch: 252/1000', 'Iteration: 1265', 'Train loss: 0.351465', 'Train acc: 0.900000')
('Epoch: 253/1000', 'Iteration: 1270', 'Train loss: 0.347819', 'Train acc: 0.908333')
('Epoch: 253/1000', 'Iteration: 1270', 'Validation loss: 0.480233', 'Validation acc: 0.856667')
('Epoch: 254/1000', 'Iteration: 1275', 'Train loss: 0.372440', 'Train acc: 0.885000')
('Epoch: 255/1000', 'Iteration: 1280', 'Train loss: 0.365679', 'Train acc: 0.888333')
('Epoch: 255/1000', 'Iteration: 1280', 'Validation loss: 0.473430', 'Validation acc: 0.863333')
('Epoch: 256/1000', 'Iteration: 1285', 'Train loss: 0.333802', 'Train acc: 0.905000')
('Epoch: 257/1000', 'Iteration: 1290', 'Train loss: 0.352546', 'Train acc: 0.886667')
('Epoch: 257/1000', 'Ite

...

('Epoch: 684/1000', 'Iteration: 3425', 'Train loss: 0.088272', 'Train acc: 0.973333')
('Epoch: 685/1000', 'Iteration: 3430', 'Train loss: 0.080267', 'Train acc: 0.981667')
('Epoch: 685/1000', 'Iteration: 3430', 'Validation loss: 0.373933', 'Validation acc: 0.915000')
('Epoch: 686/1000', 'Iteration: 3435', 'Train loss: 0.077957', 'Train acc: 0.985000')
('Epoch: 687/1000', 'Iteration: 3440', 'Train loss: 0.106590', 'Train acc: 0.968333')
('Epoch: 687/1000', 'Iteration: 3440', 'Validation loss: 0.377017', 'Validation acc: 0.913333')
('Epoch: 688/1000', 'Iteration: 3445', 'Train loss: 0.095490', 'Train acc: 0.975000')
('Epoch: 689/1000', 'Iteration: 3450', 'Train loss: 0.084757', 'Train acc: 0.976667')
('Epoch: 689/1000', 'Iteration: 3450', 'Validation loss: 0.381508', 'Validation acc: 0.913333')
('Epoch: 690/1000', 'Iteration: 3455', 'Train loss: 0.099135', 'Train acc: 0.963333')
('Epoch: 691/1000', 'Iteration: 3460', 'Train loss: 0.083024', 'Train acc: 0.976667')
('Epoch: 691/1000', 'Ite

('Epoch: 746/1000', 'Iteration: 3735', 'Train loss: 0.081255', 'Train acc: 0.976667')
('Epoch: 747/1000', 'Iteration: 3740', 'Train loss: 0.077329', 'Train acc: 0.975000')
('Epoch: 747/1000', 'Iteration: 3740', 'Validation loss: 0.378586', 'Validation acc: 0.913333')
('Epoch: 748/1000', 'Iteration: 3745', 'Train loss: 0.079966', 'Train acc: 0.981667')
('Epoch: 749/1000', 'Iteration: 3750', 'Train loss: 0.081160', 'Train acc: 0.973333')
('Epoch: 749/1000', 'Iteration: 3750', 'Validation loss: 0.381063', 'Validation acc: 0.915000')
('Epoch: 750/1000', 'Iteration: 3755', 'Train loss: 0.063837', 'Train acc: 0.986667')
('Epoch: 751/1000', 'Iteration: 3760', 'Train loss: 0.074176', 'Train acc: 0.981667')
('Epoch: 751/1000', 'Iteration: 3760', 'Validation loss: 0.380546', 'Validation acc: 0.915000')
('Epoch: 752/1000', 'Iteration: 3765', 'Train loss: 0.071966', 'Train acc: 0.975000')
('Epoch: 753/1000', 'Iteration: 3770', 'Train loss: 0.077469', 'Train acc: 0.978333')
('Epoch: 753/1000', 'Ite

('Epoch: 808/1000', 'Iteration: 4045', 'Train loss: 0.063198', 'Train acc: 0.983333')
('Epoch: 809/1000', 'Iteration: 4050', 'Train loss: 0.071033', 'Train acc: 0.983333')
('Epoch: 809/1000', 'Iteration: 4050', 'Validation loss: 0.388739', 'Validation acc: 0.916667')
('Epoch: 810/1000', 'Iteration: 4055', 'Train loss: 0.063033', 'Train acc: 0.981667')
('Epoch: 811/1000', 'Iteration: 4060', 'Train loss: 0.074941', 'Train acc: 0.978333')
('Epoch: 811/1000', 'Iteration: 4060', 'Validation loss: 0.384133', 'Validation acc: 0.915000')
('Epoch: 812/1000', 'Iteration: 4065', 'Train loss: 0.047318', 'Train acc: 0.990000')
('Epoch: 813/1000', 'Iteration: 4070', 'Train loss: 0.059995', 'Train acc: 0.986667')
('Epoch: 813/1000', 'Iteration: 4070', 'Validation loss: 0.381821', 'Validation acc: 0.915000')
('Epoch: 814/1000', 'Iteration: 4075', 'Train loss: 0.062013', 'Train acc: 0.980000')
('Epoch: 815/1000', 'Iteration: 4080', 'Train loss: 0.065772', 'Train acc: 0.985000')
('Epoch: 815/1000', 'Ite

('Epoch: 870/1000', 'Iteration: 4355', 'Train loss: 0.059072', 'Train acc: 0.985000')
('Epoch: 871/1000', 'Iteration: 4360', 'Train loss: 0.052364', 'Train acc: 0.981667')
('Epoch: 871/1000', 'Iteration: 4360', 'Validation loss: 0.393920', 'Validation acc: 0.920000')
('Epoch: 872/1000', 'Iteration: 4365', 'Train loss: 0.072274', 'Train acc: 0.980000')
('Epoch: 873/1000', 'Iteration: 4370', 'Train loss: 0.060381', 'Train acc: 0.985000')
('Epoch: 873/1000', 'Iteration: 4370', 'Validation loss: 0.392644', 'Validation acc: 0.913333')
('Epoch: 874/1000', 'Iteration: 4375', 'Train loss: 0.052808', 'Train acc: 0.985000')
('Epoch: 875/1000', 'Iteration: 4380', 'Train loss: 0.047530', 'Train acc: 0.990000')
('Epoch: 875/1000', 'Iteration: 4380', 'Validation loss: 0.392075', 'Validation acc: 0.918333')
('Epoch: 876/1000', 'Iteration: 4385', 'Train loss: 0.051433', 'Train acc: 0.983333')
('Epoch: 877/1000', 'Iteration: 4390', 'Train loss: 0.051459', 'Train acc: 0.990000')
('Epoch: 877/1000', 'Ite

('Epoch: 932/1000', 'Iteration: 4665', 'Train loss: 0.046587', 'Train acc: 0.981667')
('Epoch: 933/1000', 'Iteration: 4670', 'Train loss: 0.040895', 'Train acc: 0.991667')
('Epoch: 933/1000', 'Iteration: 4670', 'Validation loss: 0.404015', 'Validation acc: 0.916667')
('Epoch: 934/1000', 'Iteration: 4675', 'Train loss: 0.040200', 'Train acc: 0.990000')
('Epoch: 935/1000', 'Iteration: 4680', 'Train loss: 0.057595', 'Train acc: 0.978333')
('Epoch: 935/1000', 'Iteration: 4680', 'Validation loss: 0.402587', 'Validation acc: 0.916667')
('Epoch: 936/1000', 'Iteration: 4685', 'Train loss: 0.045869', 'Train acc: 0.986667')
('Epoch: 937/1000', 'Iteration: 4690', 'Train loss: 0.052606', 'Train acc: 0.980000')
('Epoch: 937/1000', 'Iteration: 4690', 'Validation loss: 0.403155', 'Validation acc: 0.916667')
('Epoch: 938/1000', 'Iteration: 4695', 'Train loss: 0.040692', 'Train acc: 0.996667')
('Epoch: 939/1000', 'Iteration: 4700', 'Train loss: 0.054910', 'Train acc: 0.981667')
('Epoch: 939/1000', 'Ite

In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []
with graph.as_default():
    prediction=tf.argmax(logits,1)
    class_probs = tf.nn.softmax(logits)
with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    y_pred = []
    y_true = []
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        batch_acc, batch_y_pred, probs = sess.run([accuracy, prediction, class_probs], feed_dict=feed)
        predict_class = [np.where(r==1)[0][0] for r in y_t ]
        y_true.extend(predict_class)
#         for idx, val in enumerate(predict_class):
#             if val == 10:
#                 print probs[idx]
        print len(predict_class)
        for idx, val in enumerate(predict_class):
            print (val, probs[idx])
        y_pred.extend(batch_y_pred)
        
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
#     print y_true
#     print y_pred
    sk_class_labels = [i for i in range(NUM_CLASS)]
    cnf_matrix = confusion_matrix(y_true, y_pred)
    print precision_recall_fscore_support(y_true, y_pred, average=None, labels=sk_class_labels)
    print 'Accuracy:', accuracy_score(y_true, y_pred)
    print 'F1 score:', f1_score(y_true, y_pred, average='micro')
    print 'Recall:', recall_score(y_true, y_pred, average='micro')
    print 'Precision:', precision_score(y_true, y_pred, average='micro')
    print '\n clasification report:\n', classification_report(y_true,y_pred, digits=5)
    print '\n confussion matrix:\n',cnf_matrix
    fig, ax = plt.subplots() 
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 20
    fig_size[1] = 10
    plt.rcParams["figure.figsize"] = fig_size
    plt.figure()
    plot_confusion_matrix(cnf_matrix, class_label_vn, title='Confusion matrix')

    plt.show()